In [1]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
#import numpy as np
import subprocess
import csv
#import scipy
import copy
import os

Function: Load the models

In [2]:
######################################## Load the models ####################################################


#load NIST 2-BTP model, with all the halogens 
#load Nora's 2-BTP model, with all the halogens 

# Load Nora's model (145 species)

full_path_Nora = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/Nora/2_BTP/FFCM_seed/2_BTP_seed/chemkin/'

Nora_chemkin_path = full_path_Nora + 'copies/copy_chem0145.inp'
Nora_dictionary_path = full_path_Nora + 'species_dictionary.txt'
Nora_transport_path = full_path_Nora + 'tran.dat'
Nora_cti_path = full_path_Nora + 'copies/copy_chem0145.cti'

Nora_species_list, Nora_reaction_list = rmgpy.chemkin.load_chemkin_file(Nora_chemkin_path, dictionary_path=Nora_dictionary_path, transport_path=Nora_transport_path)
Nora_gas = ct.Solution(Nora_cti_path)


# Load full NIST model

full_path_NIST = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/NIST/'


NIST_cti_path = full_path_NIST + '2-BTP_kinetics.cti'
NIST_chemkin_path = full_path_NIST + 'pym_gas_copy.inp'
NIST_dictionary_path = full_path_NIST + 'species_dictionary_2-BTP.txt'

NIST_gas = ct.Solution(NIST_cti_path)
NIST_dict = rmgpy.chemkin.load_species_dictionary(NIST_dictionary_path)
NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(NIST_chemkin_path, dictionary_path=NIST_dictionary_path)



**** WARNING ****
For species CH2OCH, discontinuity in cp/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  8.39347
	Value computed using high-temperature polynomial: 9.1801


**** WARNING ****
For species CH2OCH, discontinuity in h/RT detected at Tmid = 500
	Value computed using low-temperature polynomial:  42.1991
	Value computed using high-temperature polynomial: 41.9615


**** WARNING ****
For species CH2OCH, discontinuity in s/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  33.7069
	Value computed using high-temperature polynomial: 33.5121


**** WARNING ****
For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  47.6524
	Value computed using high-temperature polynomial: 48.4362


**** WARNING ****
For species C4H5-2, discontinuity in s/R detected at Tmid = 1000
	Value computed using low-temperature polynomial:  52.4292
	Value computed using high-temperature polyno

In [52]:
#define the species 
_1_reaction_list = Nora_reaction_list
_1_species_list = Nora_species_list
_2_reaction_list = NIST_reaction_list
_2_species_list = NIST_species_list

# get the mapping between RMG and NIST models
# Species Diff

common_species = []
_1_to_2_mapping = {}
_2_to_1_mapping = {}
for i, _1_sp in enumerate(_1_species_list):
    for j, _2_sp in enumerate(_2_species_list):
        if _1_sp.is_isomorphic(_2_sp):
            _1_to_2_mapping[i] = j
            _2_to_1_mapping[j] = i
            common_species.append([_1_sp, _2_sp])
            break

# Reaction Diff
common_reactions = []
_1_to_2_rxn_mapping = {}
_2_to_1_rxn_mapping = {}
for i, _1_rxn in enumerate(_1_reaction_list):
    for j, _2_rxn in enumerate(_2_reaction_list):
        if _1_rxn.is_isomorphic(_2_rxn):
            _1_to_2_rxn_mapping[i] = j
            _2_to_1_rxn_mapping[j] = i
            common_reactions.append([_1_rxn, _2_rxn])
            break
print(f'{len(common_species)} common species')
print(f'{len(common_reactions)} common reactions')

54 common species
317 common reactions


In [62]:
############################# only know the indices of the reactions we would like to flip for Nora's model. #####

equations_for_flipping_in_Noras = ['H(8) + C3H3(6822) <=> C#CC(5272)','H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23)', 'H(8) + C3H3(6822) <=> CH3(19) + C2H(22)']
index_for_flipping = []
rxns_to_change = [] #this is the list position in Nora_reaction_list


for string in equations_for_flipping_in_Noras:
    for idx,rxn in enumerate(Nora_reaction_list):
        if str(rxn)== string:
            print(rxn)
            print(idx)
            print(rxn.index)
            rxns_to_change.append(rxn.index)
            
rxns_to_change

H(8) + C3H3(6822) <=> C#CC(5272)
1271
1285
H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23)
1307
1321
H(8) + C3H3(6822) <=> CH3(19) + C2H(22)
1308
1322


[1285, 1321, 1322]

In [72]:
#################### how many reactions have radical H in Nora's model? ################

Nora_rxns_with_H = []
Nora_produces_H = []
Nora_consumes_H = []
for rxn in Nora_reaction_list: 
    for product in rxn.products: 
        if 'C3H3(6822)' == str(product):
            Nora_rxns_with_H.append(rxn)
            if rxn.is_forward==True: 
                Nora_produces_H.append(rxn)
            elif rxn.is_forward==False: 
                Nora_consumes_H.append(rxn)
                
for rxn in Nora_reaction_list: 
    for reactant in rxn.reactants: 
        if 'C3H3(6822)' == str(reactant):
            Nora_rxns_with_H.append(rxn)
            if rxn.is_forward==False: 
                Nora_produces_H.append(rxn)
            elif rxn.is_forward==True: 
                Nora_consumes_H.append(rxn)
                           
print(f'Reactions that consume H: {len(Nora_consumes_H)}\nReactions that produce H: {len(Nora_produces_H)}')



Reactions that consume H: 17
Reactions that produce H: 41


In [71]:
#################### how many reactions have radical H in NIST model? ################

NIST_rxns_with_H = []
NIST_produces_H = []
NIST_consumes_H = []
for rxn in NIST_reaction_list: 
    for product in rxn.products: 
        if 'C3H3' == str(product):
            NIST_rxns_with_H.append(rxn)
            if rxn.is_forward==True: 
                NIST_produces_H.append(rxn)
            elif rxn.is_forward==False: 
                NIST_consumes_H.append(rxn)
                
for rxn in NIST_reaction_list: 
    for reactant in rxn.reactants: 
        if 'C3H3' == str(reactant):
            NIST_rxns_with_H.append(rxn)
            if rxn.is_forward==False: 
                NIST_produces_H.append(rxn)
            elif rxn.is_forward==True: 
                NIST_consumes_H.append(rxn)
                           
print(f'Reactions that consume H: {len(NIST_consumes_H)}\nReactions that produce H: {len(NIST_produces_H)}')


Reactions that consume H: 16
Reactions that produce H: 23


In [82]:
#################### how many reactions have C3H3 in Nora's model? ################

for rxn in common_reactions: 
     if rxn[0] in Nora_rxns_with_H:        
        print(rxn[0], '\t', rxn[1])
        print(rxn[0].kinetics)
        print(rxn[0].is_forward)
        print(rxn[0].index)



HCO(17) + C3H3(6822) <=> CO(15) + C#CC(5272) 	 C3H3 + HCO <=> CO + pC3H4
Arrhenius(A=(6e+13,'cm^3/(mol*s)'), n=-0, Ea=(0.441,'kcal/mol'), T0=(1,'K'))
False
1269
CH4(3) + C3H3(6822) <=> CH3(19) + C#CC(5272) 	 CH3 + pC3H4 <=> C3H3 + CH4
Arrhenius(A=(0.02864,'cm^3/(mol*s)'), n=4.34, Ea=(19.3,'kcal/mol'), T0=(1,'K'))
False
1271
HO2(13) + C3H3(6822) <=> O2(4) + C#CC(5272) 	 C3H3 + HO2 <=> O2 + pC3H4
Arrhenius(A=(2.14707e+06,'cm^3/(mol*s)'), n=1.632, Ea=(0.473,'kcal/mol'), T0=(1,'K'))
False
1279
H(8) + C3H3(6822) <=> C#CC(5272) 	 C3H3 + H <=> pC3H4
Chebyshev(coeffs=[[13.72,0.3297,-0.05655,0.004042],[-0.6045,0.5409,-0.07255,-0.00133],[-0.4327,0.3146,-0.01216,-0.008548],[-0.2479,0.1324,0.01585,-0.005187],[-0.1203,0.03685,0.01513,0.0002071],[-0.04964,0.002288,0.00645,0.002035]], kunits='cm^3/(mol*s)', Tmin=(300,'K'), Tmax=(2500,'K'), Pmin=(0.01,'atm'), Pmax=(98.692,'atm'))
False
1285
H2(10) + C3H3(6822) <=> H(8) + C#CC(5272) 	 H + pC3H4 <=> C3H3 + H2
Arrhenius(A=(3.056,'cm^3/(mol*s)'), n=3.503,

In [ ]:
for idx, rxn in enumerate(Nora_reaction_list):